# Test library

In [ ]:
import os
import sys

# Add the parent directory to the path to find modules
sys.path.insert(0, os.path.abspath('..'))
os.chdir('..')
import earthaccess
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Login using environment variables
auth = earthaccess.login(strategy="environment")

# Search for Rome area
results = earthaccess.search_data(
    short_name="SRTMGL1",
    version="003",
    bounding_box=(12.35, 41.8, 12.65, 42.0)  # min lon, min lat, max lon, max lat
)

# Download tiles
paths = earthaccess.download(results, "./srtm_tiles")
print(paths)


# Append variables to CVS for training

In [ ]:
from shroomradar.src.topography import enrich_csv

enrich_csv(
    "..//data/Summer Bolete_climate_xy.csv",
    "..//data/Summer Bolete_climate_topography.csv",
    out_dir="dem_tiles",  # Files will be organized in subfolders: dem/, slope/, aspect/, geomorphons/
    download_tiles=True,  # 🚨 Set to True if you want to download new tiles
    variables=["dem", "slope", "aspect"],  # Specify which variables to append
)



2025-09-19 15:03:26,976 - INFO - Loading CSV: ..//data/Summer Bolete_climate_xy.csv
2025-09-19 15:03:27,034 - INFO - Loaded 4418 rows
2025-09-19 15:03:27,037 - INFO - Step 1: Scanning CSV for tile requirements...
Scanning CSV: 100%|██████████| 4418/4418 [00:00<00:00, 22212.38it/s]
2025-09-19 15:03:27,241 - INFO - Found 610 tiles that need processing
2025-09-19 15:03:27,241 - INFO - Step 2: Downloading and preparing tiles...
Preparing tiles:   0%|          | 0/610 [00:00<?, ?it/s]2025-09-19 15:03:27,991 - INFO - You're now authenticated with NASA Earthdata Login
2025-09-19 15:03:31,523 - INFO - Granules found: 1
2025-09-19 15:03:32,199 - INFO -  Getting 1 granules, approx download size: 0.0 GB
Preparing tiles:   0%|          | 1/610 [00:09<1:35:24,  9.40s/it]2025-09-19 15:03:37,892 - INFO - Granules found: 1
2025-09-19 15:03:38,212 - INFO -  Getting 1 granules, approx download size: 0.0 GB
Preparing tiles:   0%|          | 2/610 [00:15<1:13:33,  7.26s/it]2025-09-19 15:03:45,216 - INFO -

# Append to vector file

In [ ]:
from shroomradar.src.topography import enrich_geojson


enrich_geojson(
    "data/grid_tuscany_forest.geojson",
    "data/grid_tuscany_with_topography.geojson",
    out_dir="dem_tiles",
    download_tiles=True
)


# Upload to gcp

In [ ]:
from shroomradar.src.gcp import GCSManager, GCSOperationError
import logging

BUCKET_NAME = "mushroom-radar"
LOCAL_DEM_DIR = "..//dem_tiles//dem"
GCS_DEM_PREFIX = "topography/dem/"
DOWNLOAD_DIR = "dem_tiles_downloaded"



# 1. Initialize the manager with your bucket name
gcs_manager = GCSManager(BUCKET_NAME)

# 2. Upload DEM tiles to GCS
logging.info("Uploading DEM tiles to Google Cloud Storage...")
successful_uploads, failed_uploads = gcs_manager.upload_files(
    local_dir=LOCAL_DEM_DIR, gcs_prefix=GCS_DEM_PREFIX
)

# 3. List files in the bucket
logging.info("\nListing files in bucket...")
gcs_files = gcs_manager.list_files(gcs_prefix=GCS_DEM_PREFIX)
for file_path in gcs_files:
    print(f"  - {file_path}")

# 4. Download DEM tiles from GCS (example)
# logging.info("\nDownloading DEM tiles from Google Cloud Storage...")
# successful_downloads, failed_downloads = gcs_manager.download_files(
#     gcs_prefix=GCS_DEM_PREFIX, local_dir=DOWNLOAD_DIR
# )


